In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler
# %config Completer.use_jedi = False

In [42]:
import numpy as np
from pvlib import pvsystem, singlediode
import sys
module_path = "/Users/jmccarty/Data/221205_ipv_workbench/github/IPV_Workbench"
sys.path.insert(0, module_path)
from ipv_workbench.utilities import utils, time_utils

from ipv_workbench.solver import calculations as ipv_calc
import time
import pandas as pd

In [3]:
Gnull = np.zeros((264,5)) 
Geff = np.zeros((264,5)) + 500
Tcell = np.zeros((264,5)) + 25

one_day_G = np.array([Gnull] * 7 + [Geff] * 10 + [Gnull] * 7)
one_day_T = np.array([Tcell] * 24)

G_eff_ann = np.vstack([one_day_G]*365)
C_temp_ann_arr = np.vstack([one_day_T]*365)


In [4]:
submodule_map, diode_map, subcell_map = utils.read_map_excel("/Users/jmccarty/Nextcloud/Teaching/22_HS/polikseni_bano/polikseni_share/PV datasheets/final_selection/maps/cdte_landscape_maps.xlsx")
params = pd.read_csv("/Users/jmccarty/Desktop/sample_cdte_params.csv",index_col='Unnamed: 0')
param_dict = params.to_dict()[params.columns[0]]

In [38]:
module_i_dict = {}
module_v_dict = {}
module_g_dict = {}

timeseries = np.arange(0,96)

for hoy_n, hoy in enumerate(timeseries):
    Gmod = G_eff_ann[hoy_n]  
    Tmod = C_temp_ann_arr[hoy_n]  
    if np.sum(Gmod < param_dict['minimum_irradiance_cell']) > 0:
        Imod, Vmod = (np.zeros(303), np.zeros(303))
    else:
        Imod, Vmod = ipv_calc.calculate_module_curve_single_column(Geff,
                                                    Tcell, 
                                                    param_dict,
                                                    submodule_map,
                                                    diode_map,
                                                    subcell_map,
                                                    ivcurve_pnts=500)
   
    module_i_dict.update({hoy: np.round(Imod, 5)})
    module_v_dict.update({hoy: np.round(Vmod, 5)})
    module_g_dict.update({hoy: np.round(np.sum(Gmod * param_dict['one_subcell_area_m2']), 1)})
            

In [43]:
module_i_dict

{0: array([1.25745, 1.25708, 1.25672, 1.25639, 1.25606, 1.25576, 1.25547,
        1.25519, 1.25493, 1.25468, 1.25445, 1.25422, 1.25401, 1.25381,
        1.25361, 1.25343, 1.25326, 1.25309, 1.25294, 1.25279, 1.25265,
        1.25251, 1.25238, 1.25226, 1.25215, 1.25204, 1.25194, 1.25184,
        1.25175, 1.25166, 1.25157, 1.2515 , 1.25142, 1.25135, 1.25128,
        1.25122, 1.25116, 1.2511 , 1.25105, 1.251  , 1.25095, 1.2509 ,
        1.25086, 1.25082, 1.25078, 1.25074, 1.25071, 1.25067, 1.25064,
        1.25061, 1.25059, 1.25056, 1.25054, 1.25051, 1.25049, 1.25047,
        1.25045, 1.25043, 1.25042, 1.2504 , 1.25038, 1.25037, 1.25036,
        1.25034, 1.25033, 1.25032, 1.25031, 1.2503 , 1.25029, 1.25028,
        1.25028, 1.25027, 1.25026, 1.25026, 1.25025, 1.25025, 1.25024,
        1.25024, 1.25023, 1.25023, 1.25022, 1.25022, 1.25022, 1.25022,
        1.25021, 1.25021, 1.25021, 1.25021, 1.2502 , 1.2502 , 1.2502 ,
        1.2502 , 1.2502 , 1.2502 , 1.2502 , 1.2502 , 1.2502 , 1.2502 ,
   

In [40]:
module_g_dict = {}

Imod_temp = []
Vmod_temp = []

timeseries = np.arange(0,96)

for hoy_n, hoy in enumerate(timeseries):
    Gmod = G_eff_ann[hoy_n]  
    Tmod = C_temp_ann_arr[hoy_n]  
    if np.sum(Gmod < param_dict['minimum_irradiance_cell']) > 0:
        Imod, Vmod = (np.zeros(303), np.zeros(303))
    else:
        # get the first day in the array
        if hoy_n < 24:
            Imod, Vmod = ipv_calc.calculate_module_curve_single_column(Geff,
                                                        Tcell, 
                                                        param_dict,
                                                        submodule_map,
                                                        diode_map,
                                                        subcell_map,
                                                        ivcurve_pnts=500)
        # after that use only the odd hours to calculate
        elif hoy_n % 2==0:
            a = np.empty((303))
            a[:] = np.nan
            Imod = a
            Vmod = a
        else:
            Imod, Vmod = ipv_calc.calculate_module_curve_single_column(Geff,
                                                        Tcell, 
                                                        param_dict,
                                                        submodule_map,
                                                        diode_map,
                                                        subcell_map,
                                                        ivcurve_pnts=500)
        Imod_temp.append(Imod)
        Vmod_temp.append(Vmod)
        module_g_dict.update({hoy: np.round(np.sum(Gmod * param_dict['one_subcell_area_m2']), 1)})


Imod_whole = np.round(pd.DataFrame(np.array(Imod_temp)).interpolate().to_numpy(),5)
Vmod_whole = np.round(pd.DataFrame(np.array(Vmod_temp)).interpolate().to_numpy(),5)
module_i_dict.update(dict(zip(timeseries,Imod_whole)))
module_v_dict.update(dict(zip(timeseries,Vmod_whole)))
    
            

In [9]:
utils.create_sun_mask("/Users/jmccarty/Desktop/bano_simulations/shared/radiance_models/B1391/surface_1391_0_4/results/annual_irradiance/results/total/sun-up-hours.txt")[1]['HOY'].values

array([   8,    9,   10, ..., 8750, 8751, 8752])

In [12]:
file_path_sun_up_hours = "/Users/jmccarty/Desktop/bano_simulations/shared/radiance_models/B1391/surface_1391_0_4/results/annual_irradiance/results/total/sun-up-hours.txt"
sunny_df, hours = utils.create_sun_mask(file_path_sun_up_hours)
timese = pd.Series(np.arange(0,8760))

timese[hours['HOY']].head(48)

sunup_array = hours['HOY'].values

In [8]:
def set_analysis_period(sunup_array, hourly_resolution, start_hour, end_hour):
    full_hourly_timeseries = np.arange(start_hour,end_hour,1)
    sunup_timeseries = full_hourly_timeseries[sunup_array].flatten()
    analysis_timeseries = sunup_timeseries[::hourly_resolution]
    if analysis_timeseries[-1]==sunup_timeseries[-1]:
        pass
    else:
        analysis_timeseries = np.append(analysis_timeseries,sunup_timeseries[-1])
        
    return analysis_timeseries

analysis_period = set_analysis_period(sunup_array, 2, 0, 8760)
analysis_period

array([   8,   10,   12, ..., 8749, 8751, 8752])

In [18]:
ill_path = "/Users/jmccarty/Desktop/bano_simulations/shared/radiance_models/B1391/surface_1391_0_0/results/annual_irradiance/results/total/radiancegrid_1391_0_0_681sensors.ill"
ill = pd.read_csv(ill_path, delimiter=' ', header=None).iloc[:, 1:].T.reset_index(drop=True)
ill

,0,1,2,3,4,5,6,7,8,9,...,671,672,673,674,675,676,677,678,679,680
0,10.770023,10.759064,10.577964,10.703540,10.579823,10.311086,10.119455,9.281977,7.665890,5.897653,...,17.057258,17.150597,17.625861,17.079722,16.944786,17.100398,17.216241,17.147404,17.193838,17.372719
1,23.213085,23.326167,22.907167,23.151208,22.976711,22.219143,22.361755,20.590203,17.714789,15.001780,...,34.877277,34.859083,35.763036,35.119947,35.789827,35.312500,35.250369,35.206187,34.624758,35.298722
2,38.260836,38.481811,38.345437,38.401709,38.235171,37.949854,37.322322,33.723628,30.232188,26.407766,...,52.719235,52.317420,52.187244,51.847675,52.266849,52.392736,51.845476,52.291507,52.168129,52.730402
3,46.921291,46.694939,47.566845,47.105499,47.202015,47.422905,46.269360,42.477055,39.014116,34.471917,...,59.598175,58.660412,59.336605,59.049733,59.059593,58.731609,59.149391,60.174339,58.983582,59.384502
4,47.700695,47.176952,48.148777,47.272198,47.211300,46.347420,46.237305,43.610050,42.545906,38.013981,...,52.577690,52.993877,52.821060,53.036774,52.248524,53.262794,53.774960,53.016174,51.977180,52.943150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4387,48.349792,47.855923,48.737400,47.912682,47.818405,46.945179,46.746872,43.915646,42.214184,37.250038,...,54.884296,55.286625,55.122074,55.327263,54.603832,55.537636,56.011192,55.311333,54.365795,55.264133
4388,43.335533,43.554752,43.881477,43.700645,43.788925,43.198872,43.604263,40.574306,37.814465,34.586906,...,47.376255,48.068535,47.604889,47.478050,47.632832,46.922951,46.789299,46.765953,47.258881,47.207493
4389,30.476660,30.900442,30.440840,30.495008,30.399784,30.025061,29.321068,27.716780,25.024595,23.389645,...,35.335503,35.103123,34.943642,35.504673,35.396721,34.705669,34.820324,34.837345,34.777996,35.226280
4390,11.401247,11.534621,11.380877,11.316202,11.275692,11.091227,11.033762,10.007153,8.643487,7.220871,...,15.582730,15.698947,15.574902,15.529431,15.671715,15.471480,15.494483,15.518413,15.475632,15.609540


In [49]:

ill_df = utils.build_full_ill(file_path_sun_up_hours, ill_path)
ill_df

,0,1,2,3,4,5,6,7,8,9,...,671,672,673,674,675,676,677,678,679,680
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8756,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8758,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
sun_up, sun_hours = utils.create_sun_mask(file_path_sun_up_hours)

ap = time_utils.build_analysis_period(sun_hours['HOY'].values, 2)

ill_df.values[ap,:].shape

(2197, 681)

In [13]:
result_dict = {"pmp":dict(zip(np.arange(0,8760,3),[100]*int(len(np.arange(0,8760,3)))))}
result_dict

result_series = pd.Series(result_dict['pmp'] )
sun_up, sun_hours = utils.create_sun_mask(file_path_sun_up_hours)
sun_result = pd.concat([sun_hours, result_series], axis=1)
result_df = pd.merge(sun_up, sun_result, how="left", on="HOY").fillna(0)
result_df

,HOY,Sunny,0
0,0,False,0.0
1,1,False,0.0
2,2,False,0.0
3,3,False,0.0
4,4,False,0.0
...,...,...,...
8755,8755,False,0.0
8756,8756,False,0.0
8757,8757,False,0.0
8758,8758,False,0.0


In [83]:
ill_df = np.arange(0,8760,1)
sun_ill = pd.concat([sunup_array, ill_df], axis=1)
pd.merge(sun_up, sun_ill, how="left", on="HOY").fillna(0)

array([   8,    9,   10, ..., 8750, 8751, 8752])

In [105]:
sunup_arr = sun_hours['HOY'].values
sundown_arr = sun_up[sun_up['Sunny']==False]['HOY'].values

In [114]:
a = np.sort(np.arange(0,8760,1)[~sunup_array])
a
df = pd.read_csv("/Users/jmccarty/Desktop/bano_simulations/objects/B1391/results/timeseries/AP_solar_glass_2020_micro_inverter_building_level_results_hourly.csv",index_col="index")
sunup_df = df.iloc[sunup_arr].interpolate()
sundown_df = df.iloc[sundown_arr].fillna(0)

final_df = pd.concat([sunup_df,sundown_df]).sort_index()
final_df


,electricity_gen_bulk_building_kwh,self_sufficiency_building_percent,self_consumption_building_percent,electricity_specific_yield_building_kwh_kwp,electricity_gen_intensity_building_kwh,electricity_gen_bulk_1391_0_8_north_kwh,surface_capacity_1391_0_8_north_kwp,surface_area_1391_0_8_north_m2,electricity_specific_yield_1391_0_8_north_kwh_kwp,electricity_gen_intensity_1391_0_8_north_kwh_m2,...,surface_capacity_1391_0_11_west_kwp,surface_area_1391_0_11_west_m2,electricity_specific_yield_1391_0_11_west_kwh_kwp,electricity_gen_intensity_1391_0_11_west_kwh_m2,efficiency_1391_0_11_west_yield_irrad,irrad_bulk_1391_0_11_west_kWh,irrad_intensity_1391_0_11_west_kWh_m2,self_sufficiency_1391_0_11_west_percent,self_consumption_1391_0_11_west_percent,irrad_whole_building_kwh
index,,,,,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.224,1.54,0.0,0.0,...,11.829,81.327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.224,1.54,0.0,0.0,...,11.829,81.327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-01-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.224,1.54,0.0,0.0,...,11.829,81.327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-01-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.224,1.54,0.0,0.0,...,11.829,81.327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-01-01 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.224,1.54,0.0,0.0,...,11.829,81.327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 19:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.224,1.54,0.0,0.0,...,11.829,81.327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-12-31 20:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.224,1.54,0.0,0.0,...,11.829,81.327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-12-31 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.224,1.54,0.0,0.0,...,11.829,81.327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
res = utils.read_pickle("/Users/jmccarty/Desktop/building_results.pickle")

for k in res.keys():
    try:
        if len(res[k])<8760:
            print(k ,len(res[k]))
    except TypeError:
        pass

efficiency_1391_0_8_north_yield_irrad 120
irrad_bulk_1391_0_8_north_kWh 120
irrad_intensity_1391_0_8_north_kWh_m2 120
efficiency_1391_0_10_east_yield_irrad 120
irrad_bulk_1391_0_10_east_kWh 120
irrad_intensity_1391_0_10_east_kWh_m2 120
efficiency_1391_0_4_r_west_yield_irrad 120
irrad_bulk_1391_0_4_r_west_kWh 120
irrad_intensity_1391_0_4_r_west_kWh_m2 120
efficiency_1391_0_5_r_east_yield_irrad 120
irrad_bulk_1391_0_5_r_east_kWh 120
irrad_intensity_1391_0_5_r_east_kWh_m2 120
efficiency_1391_0_6_r_west_yield_irrad 120
irrad_bulk_1391_0_6_r_west_kWh 120
irrad_intensity_1391_0_6_r_west_kWh_m2 120
efficiency_1391_0_7_r_east_yield_irrad 120
irrad_bulk_1391_0_7_r_east_kWh 120
irrad_intensity_1391_0_7_r_east_kWh_m2 120
efficiency_1391_0_0_r_north_yield_irrad 120
irrad_bulk_1391_0_0_r_north_kWh 120
irrad_intensity_1391_0_0_r_north_kWh_m2 120
efficiency_1391_0_1_r_south_yield_irrad 120
irrad_bulk_1391_0_1_r_south_kWh 120
irrad_intensity_1391_0_1_r_south_kWh_m2 120
efficiency_1391_0_2_north_yield_

In [28]:
sun_up, sun_hours = utils.create_sun_mask(file_path_sun_up_hours)
sun_hours['SUNUP'] = True
sun_hours.set_index("HOY",inplace=True)

In [40]:

hoy_index = pd.Series(np.arange(0,8760,1),name='HOY')
result_df = pd.concat([hoy_index,result_series],axis=1)
result_df[0].interpolate().rename('pmp')



0       100.0
1       100.0
2       100.0
3       100.0
4       100.0
        ...  
8755    100.0
8756    100.0
8757    100.0
8758    100.0
8759    100.0
Name: pmp, Length: 8760, dtype: float64

In [54]:
from ipv_workbench.workflows import bano_workflow as bw

In [56]:
bw.main()

-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-
AP_solar_glass_2020
start solver


KeyboardInterrupt: 

In [62]:
ill_path = "/Users/jmccarty/Desktop/bano_simulations/shared/radiance_models/B1391/surface_1391_0_4/results/annual_irradiance/results/total/radiancegrid_1391_0_4_83sensors.ill"
df = pd.read_csv(ill_path, delimiter=' ', header=None).iloc[:, 1:].T.reset_index(drop=True)


,0,1,2,3,4,5,6,7,8,9,...,73,74,75,76,77,78,79,80,81,82
0,11.003121,11.511460,11.826049,12.091316,10.801022,11.294238,11.691660,11.941582,12.228842,10.566764,...,12.285662,12.404162,12.853690,12.848568,12.953485,13.246761,13.293750,13.262939,13.387531,13.347382
1,20.461069,21.570551,22.664871,23.234716,19.920675,21.363579,22.416056,22.703285,23.694462,19.387878,...,24.632656,24.975441,25.648859,25.661940,25.944029,26.421139,26.369913,26.234783,26.507116,26.461748
2,29.926357,31.087498,32.990788,34.046963,29.229809,30.944563,32.575645,33.945538,34.172115,29.000189,...,37.194023,37.813080,38.133125,38.281433,38.637188,38.083904,38.312672,38.716305,39.281338,38.833427
3,37.506947,38.512745,39.508297,40.556419,35.975426,38.115971,40.015106,40.604301,40.838196,35.122486,...,48.878613,48.663406,48.189743,47.272141,47.690281,46.832367,46.655205,45.969452,45.392998,45.463501
4,36.378127,36.230732,36.491960,35.965322,37.534005,38.313110,38.702785,37.672219,37.308605,34.343711,...,51.060540,49.532016,48.370382,46.311272,45.444679,44.667947,43.672776,43.165886,41.247063,40.177759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4387,38.171093,38.089815,38.371789,37.889613,39.274410,40.057503,40.460511,39.544490,39.201756,36.360556,...,52.469877,51.095181,50.061840,48.178324,47.390347,46.695146,45.778816,45.300633,43.523703,42.513755
4388,27.918286,28.783888,29.626831,29.591398,28.230352,28.928351,29.865246,30.755365,29.886332,28.230794,...,40.335951,37.757253,37.865982,36.662161,36.113979,36.396977,35.744962,35.172248,35.078512,35.103854
4389,21.245483,21.908594,22.253908,22.056631,21.372360,21.913214,22.055914,22.667969,22.657307,21.399193,...,24.528528,25.053333,25.139193,25.503649,25.387262,25.620907,25.678841,25.615885,25.534496,25.696230
4390,10.786661,10.873898,10.926077,10.967029,10.862801,11.000455,11.057174,11.133331,11.135151,10.979884,...,12.578555,12.691767,12.764056,12.766992,12.836475,12.790450,12.823151,12.790463,12.714140,12.631620


In [8]:

param_time = []
diode_time = []

for n in np.arange(0,100):

    submodules = np.unique(submodule_map)
    diodes = np.unique(diode_map)

    evaluated_voltages = np.linspace(0.95 * -5.5,
                                        214.3 * 1.05,
                                        1000)
    submodule_i = []
    submodule_v = []
    for submodule_key in submodules:
        _0 = time.time()
        submodule_mask = submodule_map == submodule_key
        submodule_rowmask = submodule_mask.flatten()

        submodule_diode = np.array(diode_map)[submodule_rowmask, :]
        submodule_subcell = np.array(subcell_map)[submodule_rowmask, :]

        submodule_irrad = Geff[submodule_rowmask, :]
        submodule_temp = Tcell[submodule_rowmask, :]

        diode_i = []
        diode_v = []
        _1 = time.time()
        # print("masking1", _1 - _0)
        for diode_key in diodes:
            diode_mask = submodule_diode == diode_key
            diode_rowmask = diode_mask.flatten()

            submodule_subdiode_subcell = submodule_subcell[diode_rowmask, :]

            submodule_subdiode_irrad = submodule_irrad[diode_rowmask, :]
            submodule_subdiode_temp = submodule_temp[diode_rowmask, :]

            cells_i = []
            cells_v = []
            _2 = time.time()
            # print("masking2", _2 - _1)
            for subcell_key in np.unique(submodule_subdiode_subcell):
                submodule_subdiode_subcell_mask = submodule_subdiode_subcell == subcell_key
                submodule_subdiode_subcell_irrad = submodule_subdiode_irrad[submodule_subdiode_subcell_mask]
                submodule_subdiode_subcell_temp = submodule_subdiode_temp[submodule_subdiode_subcell_mask]

                num_subcells = 10

                sde_args = np.vectorize(pvsystem.calcparams_desoto)(submodule_subdiode_subcell_irrad,
                                                                    submodule_subdiode_subcell_temp,
                                                                    alpha_sc=0.00137,
                                                                    a_ref=7.402658,
                                                                    I_L_ref=2.509123,
                                                                    I_o_ref=0.0,
                                                                    R_sh_ref=1065.831543,
                                                                    R_s=8.185414)
                _3 = time.time()
                # print("Param calc", _3 - _2)
                param_time.append(_3 - _2)
                sde_args = np.array(sde_args)
                # Use Bishop's method to calculate points on the IV curve with V ranging
                # from the reverse breakdown voltage to open circuit
                kwargs = {
                    'breakdown_factor': 0.002,
                    'breakdown_exp': 3.28,
                    'breakdown_voltage': -5.5,
                }

                i_subcell = []
                v_subcell = []


                
                for n in range(0, num_subcells):
                    i_, v_, p_ = singlediode.bishop88(evaluated_voltages,
                                                        *sde_args[:, n],
                                                        **kwargs
                                                        )
                    
                _4 = time.time()
                # print("Single Diode",_4 - _3)
                diode_time.append(_4 - _3)


IndexError: index 5 is out of bounds for axis 1 with size 5

In [6]:
print(np.sum(param_time))
print(np.sum(diode_time))

6.718991279602051
8.01072382926941


In [74]:

for t in range(0,100):
    i_subcell = []
    v_subcell = []
    for n in range(0, num_subcells):
        i_, v_, p_ = singlediode.bishop88(evaluated_voltages,
                                            *sde_args[:, n],
                                            **kwargs
                                            )
        i_ = i_ / 10
        v_ = v_ / 264
        i_subcell.append(i_)
        v_subcell.append(v_)

In [65]:
vfun = np.vectorize(singlediode.bishop88)


In [73]:
for t in range(0,100):
    i_, v_, p_ = vfun([evaluated_voltages.reshape(1000,1).tolist()],
                        sde_args[0],
                        sde_args[1],
                        sde_args[2],
                        sde_args[3],
                        sde_args[4], 
                        0,
                        np.inf,
                        kwargs['breakdown_factor'],
                        kwargs['breakdown_voltage'],
                        kwargs['breakdown_exp']
                        )
    i_subcell = i_[0] / 10
    v_subcell = v_[0] / 264
    # p_ = p_[0]

In [64]:
r[0][0]

array([[1.3477479 , 1.3477479 , 1.3477479 , ..., 1.3477479 , 1.3477479 ,
        1.3477479 ],
       [1.26868257, 1.26868257, 1.26868257, ..., 1.26868257, 1.26868257,
        1.26868257],
       [1.26007308, 1.26007308, 1.26007308, ..., 1.26007308, 1.26007308,
        1.26007308],
       ...,
       [1.14921931, 1.14921931, 1.14921931, ..., 1.14921931, 1.14921931,
        1.14921931],
       [1.14911119, 1.14911119, 1.14911119, ..., 1.14911119, 1.14911119,
        1.14911119],
       [1.14900307, 1.14900307, 1.14900307, ..., 1.14900307, 1.14900307,
        1.14900307]])

# Build Interpolation

In [ ]:
hour_1 = np.zeros((303,303)) + 